# SLCF emissions

Fossil and industrial from CEDS (1750-2019)

post-2019 non-biomass from the update to Forster et al. 2020 two year blip scenario (https://github.com/Priestley-Centre/COVID19_emissions/blob/master/global_pathways/2yearblib_0921.xlsx)

Really need update from CEDS to do this correctly, but big change (other than covid) is IMO 2020 legislation. Adjust 2019 sulfur emissions from CEDS with CAMS-GLOB-SHIP (currently available until 2022, 2023 extension requested). Assume impacts of IMO legislation on non-S species was small and can be swept up into COVID-MIP extensions.

Biomass burning from GFED (1997-2023), extended backwards to 1750 using BB4CMIP. **Note**: run `biomass-emissions.ipynb` first.

In [ ]:
import os
import numpy as np
import pandas as pd
import pooch
import h5py
import matplotlib.pyplot as pl

In [ ]:
species = ['BC', 'OC', 'SO2', 'NOx', 'CO', 'NMVOC', 'NH3']

In [ ]:
slcf_df = pd.DataFrame(columns = species, index=np.arange(1750, 2024, dtype=int))
ceds_df = pd.DataFrame(columns = species, index=np.arange(1750, 2020, dtype=int))

In [ ]:
for specie in species:
    ceds_df.loc[:, specie] = 0.001 * pd.read_csv(f'../data/slcf_emissions/ceds/{specie}_global_CEDS_emissions_by_sector_2021_04_21.csv').sum()['X1750':].values

In [ ]:
ceds_df

In [ ]:
gfed41s_df = pd.read_csv('../output/gfed4.1s_1997-2023.csv', index_col=0)

In [ ]:
gfed41s_df

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)

# RCMIP
rcmip_in_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
gfed_convert = {specie: 1 for specie in species}
gfed_convert['NOx'] = 46.006/30.006
rcmip_specie = {specie: specie for specie in species}
rcmip_specie['NMVOC'] = 'VOC'

In [ ]:
covid_df = pd.read_csv('../data/slcf_emissions/forster_et_al_2020/two_year_blip_emissions_ratios.csv', index_col=0)
covid_df

In [ ]:
for specie in species:
    rcmip_df = rcmip_in_df.loc[
        (rcmip_in_df['Scenario']=='historical')&
        (rcmip_in_df['Region']=='World')&
        (rcmip_in_df['Variable'].str.startswith(f'Emissions|{rcmip_specie[specie]}|')),
    :]
    #ceds_rcmip = [f'Emissions|{rcmip_specie}|MAGICC AFOLU|Agriculture', f'Emissions|{rcmip_specie}|MAGICC Fossil and Industrial']
    uva_rcmip = [
        f'Emissions|{rcmip_specie[specie]}|MAGICC AFOLU|Agricultural Waste Burning',
        f'Emissions|{rcmip_specie[specie]}|MAGICC AFOLU|Forest Burning',
        f'Emissions|{rcmip_specie[specie]}|MAGICC AFOLU|Grassland Burning',
        f'Emissions|{rcmip_specie[specie]}|MAGICC AFOLU|Peat Burning'
    ]
    
    
    slcf_df.loc[1750:1996, specie] = (
        ceds_df.loc[1750:1996, specie] + (
            rcmip_df.loc[rcmip_df['Variable'].isin(uva_rcmip), '1750':'1996']
            .interpolate(axis=1)
            .sum()
            .values.
            squeeze() * gfed_convert[specie]
        )
    )

    slcf_df.loc[1997:2019, specie] = (
        ceds_df.loc[1997:2019, specie] +
        gfed41s_df.loc[1997:2019, specie].values.squeeze() * gfed_convert[specie]
    )
    
    slcf_df.loc[2020:2023, specie] = (
        (ceds_df.loc[2019, specie] * covid_df.loc[2020:2023, rcmip_specie[specie]]) +
        gfed41s_df.loc[2020:2023, specie].values.squeeze() * gfed_convert[specie]
    )

In [ ]:
slcf_df.tail(7)

In [ ]:
# shipping adjustment
# what is the unit of SO4? assume SO4, and convert to SO2 units
cams_df = pd.read_csv('../data/slcf_emissions/cams/cams-glob-ship-anthro-so.csv', index_col=0)
cams_df

In [ ]:
cams_df['total'] = cams_df['SO2'] + cams_df['SO4'] * 64.066 / 96.06

In [ ]:
# according to granier et al. 2019, CAMS-GLOB-SHIP tracks only international shipping, thus we compare corresponding emissions classification
# in CEDS
ceds_full_df = pd.read_csv(f'../data/slcf_emissions/ceds/SO2_global_CEDS_emissions_by_sector_2021_04_21.csv')
ceds_ship_df = pd.DataFrame(
    ceds_full_df.loc[ceds_full_df['sector']=='1A3di_International-shipping', 'X2000':'X2019'].squeeze().values / 1000,
    index=np.arange(2000, 2020),
    columns=['SO2']
)

In [ ]:
# in recent years the datasets agree quite well, so no need to scale, but subtract 2019 offset between datasets
# we'll assume 2019 persisted from CEDS and subtract the difference from CAMS
pl.plot(ceds_ship_df)
pl.plot(cams_df['total'])

In [ ]:
offset_2019 = ceds_ship_df.loc[2019, 'SO2'] - cams_df.loc[2019, 'total']

In [ ]:
offset_2019

In [ ]:
# this is the adjustment to global SO2 to make
ceds_ship_df.loc[2019, 'SO2'] - cams_df.loc[2019:2022, 'total'] - offset_2019

In [ ]:
# a linear trend estimate to 2023 looks sensible, let's call this 8.14 TgSO2

In [ ]:
# make the adjustment
slcf_df.loc[2020:2022, 'SO2'] = slcf_df.loc[2020:2022, 'SO2'] - (ceds_ship_df.loc[2019, 'SO2'] - cams_df.loc[2019:2022, 'total'] - offset_2019)
slcf_df.loc[2023, 'SO2'] = slcf_df.loc[2023, 'SO2'] - 8.14

In [ ]:
pl.plot(slcf_df['SO2'])

In [ ]:
slcf_df['NMVOC']

In [ ]:
os.makedirs('../output', exist_ok=True)
slcf_df.to_csv('../output/slcf_emissions_1750-2023.csv')